# Minimal Integer Voting Weights Per Election
This notebook processes election data by country and date to compute minimal integer voting weights for each election. It adds two new variables per party:
- `voting_weight`: integer weight from the MWC-based algorithm
- `share_voting_weight`: that weight as a proportion of the total for that election

In [1]:
import pandas as pd
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpInteger, LpStatus, value
from itertools import combinations

# Load data
df = pd.read_excel('election_data.xlsx')

results = []
grouped = df.groupby(['countryID', 'election_date'])

def compute_voting_weights(party_seats):
    party_ids = list(party_seats.keys())
    total_seats = sum(party_seats.values())
    threshold = total_seats / 2

    def coalition_share(coalition):
        return sum(party_seats[p] for p in coalition)

    all_coalitions = []
    for r in range(1, len(party_ids)+1):
        for combo in combinations(party_ids, r):
            all_coalitions.append(set(combo))

    winning_coalitions = [c for c in all_coalitions if coalition_share(c) > threshold]

    def is_minimal(coalition):
        for r in range(1, len(coalition)):
            for subset in combinations(coalition, r):
                if coalition_share(subset) > threshold:
                    return False
        return True

    minimal_winning_coalitions = [c for c in winning_coalitions if is_minimal(c)]

    if not minimal_winning_coalitions:
        return {p: 0 for p in party_ids}, 1

    prob = LpProblem("Voting_Weights", LpMinimize)
    weights = {p: LpVariable(f"w_{p}", lowBound=0, cat=LpInteger) for p in party_ids}
    quota = LpVariable("quota", lowBound=1, cat=LpInteger)
    prob += lpSum(weights.values()), "TotalWeight"

    for mwc in minimal_winning_coalitions:
        prob += lpSum(weights[p] for p in mwc) >= quota
        for r in range(1, len(mwc)):
            for subset in combinations(mwc, r):
                prob += lpSum(weights[p] for p in subset) <= quota - 1

    prob.solve()
    result = {p: int(value(weights[p])) for p in party_ids}
    return result, sum(result.values())

# Apply voting weight computation to each election
for (countryID, election_date), group in grouped:
    party_seats = dict(zip(group['partyID'], group['seats']))
    weights, total_weight = compute_voting_weights(party_seats)
    for _, row in group.iterrows():
        pid = row['partyID']
        weight = weights.get(pid, 0)
        results.append({
            'countryID': countryID,
            'election_date': election_date,
            'partyID': pid,
            'voting_weight': weight,
            'share_voting_weight': weight / total_weight if total_weight > 0 else 0
        })

# Merge with original data
results_df = pd.DataFrame(results)
merged_df = df.merge(results_df, on=['countryID', 'election_date', 'partyID'])
merged_df.head()

,votes,share_votes,change_share_votes,seats,share_seats,change_share_seats,election_date,partyID,party_name,year,countryID,voting_weight,share_voting_weight
0,1617804,0.349,-0.079,65,0.355,-0.082,1994-10-09,at_spo01,Social Democratic Party of Austria (Sozialdem...,1994,AT,1,0.333333
1,1281846,0.277,-0.044,52,0.284,-0.044,1994-10-09,at_ovp01,Austrian People's Party (Österreichische Volk...,1994,AT,1,0.333333
2,1042332,0.225,0.059,42,0.230,0.049,1994-10-09,at_fpo01,Freedom Party of Austria (Freiheitliche Partei...,1994,AT,1,0.333333
3,338538,0.073,0.025,13,0.071,0.016,1994-10-09,at_ga01,The Greens-Green Alternative (Die Grünen-Die G...,1994,AT,0,0.000000
4,276580,0.060,0.060,11,0.060,0.033,1994-10-09,at_lif01,"Liberal Forum (Liberales Forum, LIF)",1994,AT,0,0.000000


In [2]:
merged_df.to_excel("election_data_weights.xlsx", index=False)